# Lesson: Introduction to User Authentication with Node.js and Express.js

### Topic Overview and Introduction

Hello! Today, we're unraveling a cornerstone of web application security - User Authentication. User Authentication involves verifying user identities during login attempts. By correctly implementing User Authentication, you can significantly protect your applications from unauthorized access and potential security threats. Our toolkit for today includes Node.js and Express.js for crafting our server-side application, and MongoDB with Mongoose for managing our users' data.

### Creating User Models with MongoDB and Mongoose

Next, we'll delve into MongoDB, a powerful NoSQL database, and Mongoose, a MongoDB object modeling tool designed to work in an asynchronous environment. They'll assist us in storing and managing user data in a structured manner.

Let's dive straight in and create a User Model, comprising username and password attributes:

In [ ]:
var mongoose = require('mongoose'); // Importing the Mongoose library
var Schema = mongoose.Schema; // Retrieving the Schema object from Mongoose
var UserSchema = new Schema({
  username: String, // Setting the username attribute as a string
  password: String, // Setting the password attribute as a string
});
var User = mongoose.model('User', UserSchema); // Creating a User model with the above schema

Here, mongoose.model is used to create a User model in our MongoDB database. Each User document in our database will have a username and password field.

### Implementing Authentication Middleware with Express.js

Now, let's apply the final touches and create our authentication middleware. Middleware in Express provides a way to work with request and response objects in your application. Middleware functions can perform tasks such as modifying these objects, ending the request-response cycle, or invoking the next middleware function in the stack.

Here's a basic authentication middleware function:

In [ ]:
var authMiddleware = function (req, res, next) {
  if (req.query.username === "admin" && req.query.password === "admin") {
   // If the user is authenticated, the next function is invoked
    console.log("User authenticated");
    next();
  } else {
   // If credentials are invalid, we respond with 'Invalid credentials'
    res.status(401).send({ message: "Invalid credentials" });
  }
};

This authMiddleware checks whether the submitted username and password match the known credentials. If they match, the next function in the middleware stack is invoked; otherwise, the response 'Invalid credentials' is sent.

### Improving Middleware Authenticaton with MongoDB and Mongoose

We can further improve our authentication middleware by using MongoDB to store and retrieve user credentials, making our method more robust and practical.

Instead of hardcoding a username and password into our middleware function, we can query the user’s data from the MongoDB database using Mongoose. A successful database query will mean the user details are valid, and the user is authenticated.

To handle the asynchronous query calls in an efficient, readable way, we'll use JavaScript's native async/await syntax. This approach leads to cleaner and more structured code, avoiding the potential pitfalls of callback functions.

Let's see how the authentication middleware looks when we use MongoDB and Mongoose:

In [ ]:
var authMiddleware = async function (req, res, next) {
  // Search for the user credentials in the MongoDB database
  var user = await User.findOne({ username: req.query.username, password: req.query.password });

  // If the query returned a user, proceed with the next function
  if (user) {
    console.log("User authenticated");
    next();
  } else {
    // If no user returned from the query, authentication fails 
    res.status(401).send({ message: "Invalid credentials" });
  }
};

In this updated version of authMiddleware, we fetch the user data with a Mongoose query inside an async function, and then await the results. The query returns the user's document if the username and password in the request match a document in the MongoDB database. Otherwise, it returns null.

If the user exists, next() is called to proceed with the route handlers. If no user is returned (indicating the credentials don't match any document in the database), a response with 'Invalid credentials' is sent instead.

### Lesson Summary and Practice

Fantastic work! We've explored User Authentication, constructed a user model in MongoDB using Mongoose, and established an authentication middleware using Express.js.

It's now time to put our learning into practice! This will not only reinforce your understanding of the new concepts but also demonstrate them in action by simulating real-world web development scenarios. Remember, practice is the cornerstone of mastering any new concept. Best of luck!


Have you ever wondered how an online bookstore handles user logins? The given code illustrates a simple server setup using Express and MongoDB that validates user login attempts. Let's see it in action! Click Run to observe the backend authentication process when a user attempts to log in.

In [ ]:
// index.js (Express server)

const express = require('express');
const mongoose = require('mongoose');
const User = require('./models/User');
const cors = require('cors');

const app = express();
app.use(express.json());
app.use(cors());

mongoose
.connect('mongodb://127.0.0.1:27017/bookstore', { useNewUrlParser: true, useUnifiedTopology: true })
.then(() => {
    console.log('Connected to the MongoDB database.');
})
.catch((err) => {
    console.error('Error connecting to the database', err);
});

// Let's add one user to our DB 
User.create({
  username: 'bookworm',
  password: 'read1234'
})
.then((doc) => console.log('User created:', doc))
.catch((err) => console.error('Error creating user:', err));

function validatePassword(userPassword, inputPassword) {
  return userPassword === inputPassword;
}

app.post('/api/login', async (req, res) => {
    const { username, password } = req.body;
    try {
        const user = await User.findOne({ username: username });
        if (!user) {
           res.status(404).send({ message: 'User not found' });
        }
        if (!validatePassword(user.password, password)) {
           res.status(401).send({ message: 'Invalid password' });
        }
        res.status(200).send({ message: 'User logged in successfully' });
    } catch (err) {
       res.status(500).send({ message: 'Internal Server Error' });
    }
});

app.listen(5000, () => {
  console.log('Server running on port 5000');
});

In [ ]:
// App.js

import { useState } from 'react';
import axios from 'axios';

const App = () => {
  const [credentials, setCredentials] = useState({ username: 'bookworm', password: 'read1234' });
  const [message, setMessage] = useState('');

  const handleLogin = () => {
    axios.post(`/api/login`, credentials)
      .then(response => setMessage(response.data.message))
      .catch(error => setMessage(error.response.data.message));
  };
  
  const handleChange = event => {
    setCredentials({ ...credentials, [event.target.name]: event.target.value });
  };
  
  return (
    <div>
      <input
        name="username"
        type="text"
        value={credentials.username}
        onChange={handleChange}
      />
      <input
        name="password"
        type="password"
        value={credentials.password}
        onChange={handleChange}
      />
      <button onClick={handleLogin}>Login</button>
      {message && <p>{message}</p>}
    </div>
  );
};

export default App;

In [ ]:
// user.js (React LoginForm Component)
const mongoose = require('mongoose');
const Schema = mongoose.Schema;

const UserSchema = new Schema({
  username: { type: String, required: true },
  password: { type: String, required: true }
});

module.exports = mongoose.model('User', UserSchema);

Stellar Navigator, let's ensure that our authentication middleware in server/index.js uses Mongoose to check both username and password securely.

Update the authMiddleware to query the User model for a match with req.query.username and req.query.password. Keep those galactic credentials secure!

In [ ]:
//index.js (Express server with Authentication Middleware)

const express = require('express');
const mongoose = require('mongoose');

const userSchema = new mongoose.Schema({
  username: String,
  password: String,
});

User = mongoose.model('User', userSchema);

const app = express();
const port = 5000;

// Connect to MongoDB and create one user
mongoose
    .connect('mongodb://127.0.0.1:27017/bookstoredb', { useNewUrlParser: true, useUnifiedTopology: true })
    .then(() => {
        console.log('Connected to the MongoDB database.');
      User.create({
        username: 'admin',
        password: 'admin'
      })
    })
    .catch((err) => {
        console.error('Error connecting to the database', err);
    });

// Authentication Middleware
const authMiddleware = async function (req, res) {
  try {
    // TODO: Update the query to check both username and password
    const user = await User.findOne({ username: req.query.username, password: req.query.password }); // added ", password: req.query.password" to the query
    
    if (user) {
      res.send("Welcome to the Online Bookstore!");
    } else {
      res.status(401).send({ message: 'Invalid credentials' });
    }
  } catch (error) {
    console.error('Authentication error:', error);
    res.status(500).send("Authentication failed due to an error");
  }
};

// Login Route
app.get('/api/login', authMiddleware);

app.listen(port, () => {
  console.log(`Bookstore server running on port ${port}`);
});

In [ ]:
// App.js (React Component)
import React, { useState } from 'react';
import axios from 'axios';

const App = () => {
  const [loginStatus, setLoginStatus] = useState('');

  const checkLogin = () => {
    axios.get('/api/login?username=admin&password=admin')
      .then(response => {
        setLoginStatus(response.data);
      })
      .catch(error => {
        console.error('Error during login attempt:', error);
        setLoginStatus('Login failed');
      });
  };
  
  return (
    <div>
      <button onClick={checkLogin}>Login as Admin</button>
      <p>{loginStatus}</p>
    </div>
  );
};

export default App;

You've been rocketing through this, Stellar Navigator! Now it's time to secure the command deck. Implement the missing authentication code to ensure that only authorized personnel can access the welcome message.

In [ ]:
// index.js (Express server)
const express = require('express');
const cors = require('cors');
const app = express();

app.use(cors());
app.use(express.json());

const hardcodedAuth = (req, res, next) => {
  // TODO: Compare req.body.username and req.body.password with hardcoded values { username='admin', password='admin' }
  // Call next() if authenticated, else respond with status 401 and 'Invalid credentials'
  if (req.body.username === "admin" && req.body.password === "admin") {
    console.log("User authenticated");
    next();
  } else {
    // If no user returned from the query, authentication fails 
    res.status(401).send({ message: "Invalid credentials" });
  };
};

app.post('/api/login', hardcodedAuth, (req, res) => {
  // Server should send back a welcome message if the user is authenticated
  res.send("Welcome, user. You're authenticated");
  
});

app.listen(5000, () => console.log("Server running on port 5000"));

In [ ]:
// App.js
function App() {
  let message = '';

  const login = () => {
    const credentials = { username: 'admin', password: 'admin' };
    fetch('/api/login', { 
      method: 'POST', 
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify(credentials)
    })
    .then(response => response.text())
    .then(data => message = data)
    .catch(error => message = error.message);

    document.getElementById('response').textContent = message;
  };

  return (
    <div>
      <button onClick={login}>Log In</button>
      <div id="response"></div>
    </div>
  );
}

export default App;

Stellar Navigator, let's continue shaping our Online Bookstore's space. Implement the missing logic in the server code to check a user's login credentials. You'll need to find the user in the database and verify the password. Engage!

In [ ]:
// index.js (Express server)
const express = require('express');
const mongoose = require('mongoose');
const User = require('./models/User');

const app = express();
const port = 5000;

// Connecting to MongoDB and creating one user
mongoose.connect('mongodb://127.0.0.1:27017/bookstore', { useNewUrlParser: true, useUnifiedTopology: true })
    .then(async () => {
        console.log('Connected to the MongoDB database.');

        // Attempt to create a user
        try {
            const newUser = await User.create({
                username: 'booklover',
                password: 'read123' // In a real application, passwords should be hashed
            });
            console.log('User created successfully:', newUser);
        } catch (err) {
            if (err.code === 11000) {
                // Handle duplicate key error (e.g., username already exists)
                console.log('User already exists.');
            } else {
                // Handle other possible errors
                console.error('Error creating the user:', err);
            }
        }
    })
    .catch((err) => {
        console.error('Error connecting to the database', err);
    });

// Login route
app.get('/api/login', async (req, res) => {
  const { username, password } = req.query;
  
  // TODO: Find the user by username in the database
  try {
        const user = await User.findOne({ username: username });
        if (!user) {
           res.status(404).send({ message: 'User not found' });
        }
  // TODO: Verify if the found user's password matches the provided password
        if (user.password === password) {
           res.status(200).send({ message: 'User logged in successfully' });
        }
        res.status(401).send({ message: 'Invalid password' });
    } catch (err) {
       res.status(500).send({ message: 'Internal Server Error' });
    }
  
});

app.listen(port, () => {
  console.log(`Server running on port ${port}`);
});

In [ ]:
// App.js
import { useState } from 'react';
import axios from 'axios';

const App = () => {
  const [loginStatus, setLoginStatus] = useState('');

  const handleLogin = () => {
    const username = 'booklover';
    const password = 'read123';
    axios.get(`/api/login?username=${username}&password=${password}`)
      .then(response => {
        setLoginStatus(response.data);
      })
      .catch(error => {
        console.log(error);
        setLoginStatus('Login failed. Please try again.');
      });
  };

  return (
    <div>
      <button onClick={handleLogin}>Login</button>
      <p>{loginStatus}</p>
    </div>
  );
};

export default App;

In [ ]:
// User.js

const mongoose = require('mongoose');
const Schema = mongoose.Schema;

const UserSchema = new Schema({
  username: { type: String, required: true },
  password: { type: String, required: true }
});

module.exports = mongoose.model('User', UserSchema);

Alright, Space Voyager, it's time to cement your grasp of user authentication in our Online Bookstore universe. Can you set up the backend and frontend to manage user logins? Use your knowledge of backend authentication and frontend state handling to authenticate our admin user. Then, display a welcome message if their login is successful. Are you ready to embark on your final mission for this lesson?

In [ ]:
// App.js
import { useState } from 'react';
import axios from 'axios';

function App() {
  // TODO: Set the initial state for the loggedIn variable using useState
  const [loginStatus, setLoginStatus] = useState('');

  const handleLogin = () => {
    // TODO: Send a GET request to the backend /api/login endpoint to check admin credentials with query params: username: 'admin', password: 'admin'
    const credentials = {username: 'admin', password: 'admin'};
    axios.get(`/api/login?username=${username}&password=${password}`)
      .then(response => {
        setLoginStatus(response.data);
      })
      .catch(error => {
        console.log(error);
        setLoginStatus('Login failed. Please try again.');
      });
  };

  // TODO: Return JSX including a 'Login as Admin' button and a welcome message if logged in
  return (
    <div>
      <button onClick={handleLogin}>Login as Admin</button>
      <p>{loginStatus}</p>
    </div>
  );
}

export default App;

In [ ]:
//index.js (Express server with Authentication Middleware)
const express = require('express');
const app = express();
const mongoose = require('mongoose');
const User = require('./models/User');

// TODO: Set up your middleware and route for handling the login logic
mongoose.connect('mongodb://127.0.0.1:27017/bookstore', { useNewUrlParser: true, useUnifiedTopology: true })
    .then(async () => {
        console.log('Connected to the MongoDB database.');
        
var authMiddleware = async function (req, res, next) {
  // Search for the user credentials in the MongoDB database
  var user = await User.findOne({ username: req.query.username, password: req.query.password });

  // If the query returned a user, proceed with the next function
  if (user) {
    console.log("User authenticated");
    next();
  } else {
    // If no user returned from the query, authentication fails 
    res.status(401).send({ message: "Invalid credentials" });
  }
};

app.use('/api/login', authMiddleware); // TODO: Update to include your middleware and send a response on successful authentication

app.listen(5000, () => {
    console.log(`Server is running on Port 5000`);
}); // TODO: Add a console log to indicate that the server is running